## 1. Setup and Imports {#setup}

Let's start by importing all necessary modules and setting up the environment.

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import sys
import os
warnings.filterwarnings('ignore')

import sys
import os
sys.path.insert(0, '/home/HX46_FR5/repo_perso/AbXtract')


from AbXtract import *
from AbXtract import AntibodyDescriptorCalculator, Config, load_config
from AbXtract.sequence import (
    SequenceLiabilityAnalyzer,
    BashourDescriptorCalculator,
    PeptideDescriptorCalculator,
    AntibodyNumbering
)
from AbXtract.structure import (
    SASACalculator,
    ChargeAnalyzer,
    DSSPAnalyzer,
    PropkaAnalyzer,
    ArpeggioAnalyzer
)
from AbXtract.utils import (
    read_fasta,
    write_fasta,
    parse_sequence,
    validate_sequence
)
    

# Load config

In [2]:
# default configuration
custom_config = Config()

'''
# Test custom configuration
custom_config = Config.from_dict({
    'pH': 7.4,
    'numbering_scheme': 'kabat',
    'verbose': True,
    'calculate_dssp': tool_status.get('dssp', False),
    'calculate_propka': tool_status.get('propka', False),
    'calculate_arpeggio': tool_status.get('arpeggio', False)
})
'''


# Check external tool availability
tool_status = custom_config.check_external_tools()
print("🛠️ External Tool Status:")
for tool, available in tool_status.items():
    status = "✅" if available else "❌"
    print(f"  {tool}: {status}")


reduce not found at reduce


🛠️ External Tool Status:
  dssp: ✅
  propka: ✅
  arpeggio: ✅
  reduce: ❌
  muscle: ✅


# Load classes

In [3]:
numbering = AntibodyNumbering(scheme='imgt')
peptide_calc = PeptideDescriptorCalculator()
calc = AntibodyDescriptorCalculator(config=custom_config)

reduce not found at reduce
Missing external tools: reduce
Some analyses may not be available


# Define path 

In [4]:
abxtract_path = "/home/HX46_FR5/repo_perso/AbXtract"
sys.path.insert(0, abxtract_path)

# Set up test data paths
BASE_DIR = Path.cwd() 
DATA_DIR = BASE_DIR / "data" / "test"
DATA_DIR.mkdir(parents=True, exist_ok=True)


# Define test file paths
RESULTS_DIR = DATA_DIR / "results"
RESULTS_DIR.mkdir(exist_ok=True)


# Input sequence and pdb

In [5]:
# Test antibody sequences (based on therapeutic antibodies)
HEAVY_SEQUENCE = (
    "QVQLVQSGAEVKKPGASVKVSCKASGGTFSSYAISWVRQAPGQGLEWMGGIIPIFGTANYAQKFQGRVTITADESTSTAYMELSSLRSEDTAVYYCARSHYGLDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYASTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK"
)
# Light chain: Includes realistic VL domain + human kappa constant region  
LIGHT_SEQUENCE = (
    "DIQMTQSPSSLSASVGDRVTITCRASHSISSYLAWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYSTPLTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC"
)
PDB_FILE = DATA_DIR / "test.pdb"  # User will provide this

# Sequence validity for numbering

In [6]:
heavy_valid, heavy_msg = validate_sequence(HEAVY_SEQUENCE)
light_valid, light_msg = validate_sequence(LIGHT_SEQUENCE)


# A. Numbering

In [7]:
heavy_numbered = numbering.number_sequence(HEAVY_SEQUENCE, 'H')  # Use VH portion only
light_numbered = numbering.number_sequence(LIGHT_SEQUENCE, 'L')  # Use VH portion only

annotated_H, cdrs_H = numbering.get_cdr_sequences(heavy_numbered, 'H')
annotated_L, cdrs_L = numbering.get_cdr_sequences(light_numbered, 'L')

heavy_profiles = numbering.get_peptide_profiles(HEAVY_SEQUENCE)
light_profiles = numbering.get_peptide_profiles(LIGHT_SEQUENCE)

# B. Peptide descriptors

In [8]:
peptide_results = peptide_calc.calculate_all(
    heavy_sequence=HEAVY_SEQUENCE,
    light_sequence=LIGHT_SEQUENCE
)


# C. Sequence descriptors

In [9]:
sequence_results, liabilities = calc.calculate_sequence_descriptors(
    heavy_sequence=HEAVY_SEQUENCE,
    light_sequence=LIGHT_SEQUENCE,
    sequence_id="TestAb_Sequence"
)

# D. Sequence descriptors

In [10]:


# Run structure analysis if PDB is available
structure_results_seq, structure_results_comp, df_residues = calc.calculate_structure_descriptors(
    pdb_file=PDB_FILE,
    structure_id="TestAb_Structure"
)




@> 1818 atoms and 1 coordinate set(s) were parsed in 0.02s.
DSSP analysis failed: DSSP failed to produce an output


# Organise outputs

In [11]:
heavy_valid, light_valid

(True, True)

In [12]:
heavy_numbered, light_numbered

([((1, ' '), 'Q'),
  ((2, ' '), 'V'),
  ((3, ' '), 'Q'),
  ((4, ' '), 'L'),
  ((5, ' '), 'V'),
  ((6, ' '), 'Q'),
  ((7, ' '), 'S'),
  ((8, ' '), 'G'),
  ((9, ' '), 'A'),
  ((11, ' '), 'E'),
  ((12, ' '), 'V'),
  ((13, ' '), 'K'),
  ((14, ' '), 'K'),
  ((15, ' '), 'P'),
  ((16, ' '), 'G'),
  ((17, ' '), 'A'),
  ((18, ' '), 'S'),
  ((19, ' '), 'V'),
  ((20, ' '), 'K'),
  ((21, ' '), 'V'),
  ((22, ' '), 'S'),
  ((23, ' '), 'C'),
  ((24, ' '), 'K'),
  ((25, ' '), 'A'),
  ((26, ' '), 'S'),
  ((27, ' '), 'G'),
  ((28, ' '), 'G'),
  ((29, ' '), 'T'),
  ((30, ' '), 'F'),
  ((35, ' '), 'S'),
  ((36, ' '), 'S'),
  ((37, ' '), 'Y'),
  ((38, ' '), 'A'),
  ((39, ' '), 'I'),
  ((40, ' '), 'S'),
  ((41, ' '), 'W'),
  ((42, ' '), 'V'),
  ((43, ' '), 'R'),
  ((44, ' '), 'Q'),
  ((45, ' '), 'A'),
  ((46, ' '), 'P'),
  ((47, ' '), 'G'),
  ((48, ' '), 'Q'),
  ((49, ' '), 'G'),
  ((50, ' '), 'L'),
  ((51, ' '), 'E'),
  ((52, ' '), 'W'),
  ((53, ' '), 'M'),
  ((54, ' '), 'G'),
  ((55, ' '), 'G'),
  ((56, '

In [13]:
cdrs_H, cdrs_L


({'cdrh1': 'GGTFSSYA', 'cdrh2': 'IIPIFGTA', 'cdrh3': 'ARSHYGLDY'},
 {'cdrl1': 'HSISSY', 'cdrl2': 'AAS', 'cdrl3': 'QQSYSTPLT'})

In [14]:
annotated_H, annotated_L

([((1, ' '), 'Q', 'fwh1'),
  ((2, ' '), 'V', 'fwh1'),
  ((3, ' '), 'Q', 'fwh1'),
  ((4, ' '), 'L', 'fwh1'),
  ((5, ' '), 'V', 'fwh1'),
  ((6, ' '), 'Q', 'fwh1'),
  ((7, ' '), 'S', 'fwh1'),
  ((8, ' '), 'G', 'fwh1'),
  ((9, ' '), 'A', 'fwh1'),
  ((11, ' '), 'E', 'fwh1'),
  ((12, ' '), 'V', 'fwh1'),
  ((13, ' '), 'K', 'fwh1'),
  ((14, ' '), 'K', 'fwh1'),
  ((15, ' '), 'P', 'fwh1'),
  ((16, ' '), 'G', 'fwh1'),
  ((17, ' '), 'A', 'fwh1'),
  ((18, ' '), 'S', 'fwh1'),
  ((19, ' '), 'V', 'fwh1'),
  ((20, ' '), 'K', 'fwh1'),
  ((21, ' '), 'V', 'fwh1'),
  ((22, ' '), 'S', 'fwh1'),
  ((23, ' '), 'C', 'fwh1'),
  ((24, ' '), 'K', 'fwh1'),
  ((25, ' '), 'A', 'fwh1'),
  ((26, ' '), 'S', 'fwh1'),
  ((27, ' '), 'G', 'cdrh1'),
  ((28, ' '), 'G', 'cdrh1'),
  ((29, ' '), 'T', 'cdrh1'),
  ((30, ' '), 'F', 'cdrh1'),
  ((35, ' '), 'S', 'cdrh1'),
  ((36, ' '), 'S', 'cdrh1'),
  ((37, ' '), 'Y', 'cdrh1'),
  ((38, ' '), 'A', 'cdrh1'),
  ((39, ' '), 'I', 'fwh2'),
  ((40, ' '), 'S', 'fwh2'),
  ((41, ' '), 'W', 'f

In [15]:
heavy_profiles, light_profiles


({'charge_sign': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -1.0,
   0.0,
   1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   -1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -1.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -1.0,
   -1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -1.0,
   0.0,
   -1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   -1.0,
   -1.0,
   0.0,
   0.0,
   0.0,
   -1.0,
   -1.0,
   -1.0,
   0.0,
   1.0,
   0.0,
   1.0,
   -1.0,
   0.0,
   0.0,
   -1.0,
   -1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,


In [16]:
liabilities

,SeqID,Type,Heavy_Length,Light_Length,liabilities
0,TestAb_Sequence,Antibody,446,214,"[{'name': 'Met oxidation (M)', 'chain': 'L', '..."


In [17]:
peptide_results

,chain,sequence,length,aliphatic_index,boman,charge_pH7.4,isoelectric_point,instability_index,molecular_weight,molar_extinction_reduced,...,protpy_apaac_APAAC_72,protpy_apaac_APAAC_73,protpy_apaac_APAAC_74,protpy_apaac_APAAC_75,protpy_apaac_APAAC_76,protpy_apaac_APAAC_77,protpy_apaac_APAAC_78,protpy_apaac_APAAC_79,protpy_apaac_APAAC_80,hydrophobic_moment_w5_a100
0,Heavy,QVQLVQSGAEVKKPGASVKVSCKASGGTFSSYAISWVRQAPGQGLE...,446,71.591928,1.358004,3.765413,8.321377,46.153363,48473.79784,NaN,...,0.802,0.633,-1.139,0.633,0.591,-0.886,-1.941,-0.169,-0.211,0.864565
1,Light,DIQMTQSPSSLSASVGDRVTITCRASHSISSYLAWYQQKPGKAPKL...,214,67.429907,1.643505,0.375036,7.847862,57.210748,23173.78954,NaN,...,-0.172,-0.560,0.043,-0.129,0.129,2.543,0.043,-3.276,-2.198,1.029839


In [18]:
sequence_results

,SeqID,Type,Heavy_Length,Light_Length,count,total_possible,heavy_sequence,light_sequence,scheme,Heavy_Molecular Weight,...,Light_Charges_pH_5,Light_Charges_pH_6,Light_Charges_pH_7,Light_Charges_pH_8,Light_Charges_pH_9,Light_Charges_pH_10,Light_Charges_pH_11,Light_Charges_pH_12,Light_Charges_pH_13,Light_Charges_pH_14
0,TestAb_Sequence,Antibody,446,214,4,12,QVQLVQSGAEVKKPGASVKVSCKASGGTFSSYAISWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASHSISSYLAWYQQKPGKAPKL...,imgt,48473.79784,...,5.469652,2.663232,0.968473,-1.100358,-4.668085,-12.176418,-23.796527,-28.944735,-32.55211,-33.81848


In [19]:
structure_results_seq

,disulfide_bonds,residue_sap,high_sap_residues,residue_sasa,sidechain_sasa,relative_sasa,buried_residues,residue_pka,pka_shifts,charge_profile,stability_profile
0,"[{'cys1': 'H_21', 'cys2': 'H_95', 'distance': ...","{'ASP_0_H': 0.029531839957845273, 'VAL_1_H': 0...","[LEU_10_H, PRO_40_H, TYR_58_H, LEU_101_H, TYR_...","{'ASP_0_H': {'N': 40.450005754993995, 'CA': 9....","{'ASP_0_H': 111.79910841184281, 'VAL_1_H': 31....",{},"[LEU_3_H, GLU_5_H, LEU_19_H, CYS_21_H, VAL_28_...","0 {'ASP_54_H': 2.55, 'ASP_61_H': 2.76, 'ASP...","0 {'ASP_54_H': -1.25, 'ASP_61_H': -1.04, 'A...","0 {0.0: {'folded': 11.99, 'unfolded': 12.0}...","0 {0.0: 11.82, 1.0: 11.76, 2.0: 11.18, 3.0:..."


# Format standard

### 1. Residue annotation

In [20]:
# Creating comprehensive heavy chain dataframe
def create_comprehensive_df(annotations, hydrophobicity, chain_type='Heavy'):
    # Start with basic annotation data
    data = []
    
    for item in annotations:
        position_tuple, amino_acid, region = item
        position_num = position_tuple[0]
        
        # Get index for hydrophobicity values (0-based)
        idx = position_num - 1
        
        # Create row with all information
        row = {
            'position': position_num,
            'amino_acid': amino_acid,
            'region': region,
            'charge_sign': hydrophobicity['charge_sign'][idx] if idx < len(hydrophobicity['charge_sign']) else np.nan,
            'hydrophobicity_hw': hydrophobicity['hydrophobicity_hw'][idx] if idx < len(hydrophobicity['hydrophobicity_hw']) else np.nan,
            'hydrophobicity_eisenberg': hydrophobicity['hydrophobicity_eisenberg'][idx] if idx < len(hydrophobicity['hydrophobicity_eisenberg']) else np.nan,
            'hydrophobicity_rose': hydrophobicity['hydrophobicity_rose'][idx] if idx < len(hydrophobicity['hydrophobicity_rose']) else np.nan,
            'hydrophobicity_janin': hydrophobicity['hydrophobicity_janin'][idx] if idx < len(hydrophobicity['hydrophobicity_janin']) else np.nan,
            'hydrophobicity_engelman': hydrophobicity['hydrophobicity_engelman'][idx] if idx < len(hydrophobicity['hydrophobicity_engelman']) else np.nan
        }
        data.append(row)
    
    return pd.DataFrame(data)

import pandas as pd
import numpy as np

def add_liability_columns(df, chain_type, liabilities_list):
    """
    Add boolean columns for each LIABILITY TYPE (not just the ones present in this chain).
    Place these columns BEFORE the position column.
    """
    chain_letter = 'H' if chain_type == 'Heavy' else 'L'
    
    # Define ALL possible liability types based on your liability definitions
    all_liability_types = [
        'Unpaired_Cys',
        'N-linked_glycosylation',
        'Met_oxidation',
        'Trp_oxidation',
        'Asn_deamidation',
        'Asp_isomerisation',
        'Lysine_Glycation',
        'N-terminal_glutamate',
        'Integrin_binding',
        'CD11c/CD18_binding',
        'Fragmentation',
        'Polyreactivity'
    ]
    
    # Initialize ALL liability columns as False
    for col_name in all_liability_types:
        df[col_name] = False
    
    # Now mark positions that have each liability based on the actual data
    for liability in liabilities_list:
        if liability['chain'] == chain_letter:
            # Get position range
            start_pos = liability['start_position'][0]
            end_pos = liability['end_position'][0]
            
            # Create column name by simplifying the liability name
            col_name = liability['name'].split('(')[0].strip().replace(' ', '_').replace('/', '')
            
            # Mark all positions in range as True
            mask = (df['position'] >= start_pos) & (df['position'] <= end_pos)
            if mask.any():
                df.loc[mask, col_name] = True
    
    # Reorder columns: liability columns FIRST, then position, amino_acid, region, then hydrophobicity
    liability_cols = all_liability_types
    base_cols = ['position', 'amino_acid', 'region']
    hydro_cols = ['charge_sign', 'hydrophobicity_hw', 'hydrophobicity_eisenberg', 
                  'hydrophobicity_rose', 'hydrophobicity_janin', 'hydrophobicity_engelman']
    
    # New column order: liabilities first, then base, then hydrophobicity
    new_order = base_cols + hydro_cols + liability_cols
    df = df[new_order]
    
    return df


import pandas as pd
import numpy as np

def add_residue_sasa_sum_column(structures_df):
    """
    Add a column with the sum of residue SASA per position to structures_results_seq dataframe.
    
    Parameters:
    structures_df: DataFrame with columns including 'sidechain_sasa' containing SASA dictionaries
    
    Returns:
    DataFrame with added 'residue_sasa_sum' column
    """
    
    # Initialize the new column as a list to hold dictionaries
    residue_sasa_sum_list = []
    
    # Process each row
    for idx, row in structures_df.iterrows():
        # Get the sidechain_sasa dictionary for this row
        sidechain_sasa = row['sidechain_sasa']
        
        # Create a new dictionary for residue_sasa_sum
        # The key will be the position number, value will be the SASA sum
        residue_sasa_sum = {}
        
        if isinstance(sidechain_sasa, dict):
            # Process each residue in the sidechain_sasa dictionary
            for residue_key, sasa_value in sidechain_sasa.items():
                # Extract position from the residue key (e.g., 'ASP_0_H' -> 0)
                # Split by underscore and get the position (middle part)
                parts = residue_key.split('_')
                if len(parts) >= 3:
                    try:
                        # Position in the key is 0-based, convert to 1-based for consistency
                        position = int(parts[1]) + 1
                        
                        # Store the SASA sum with position as key
                        residue_sasa_sum[position] = sasa_value
                    except ValueError:
                        # Skip if position is not a valid integer
                        continue
        
        # Append the dictionary to the list
        residue_sasa_sum_list.append(residue_sasa_sum)
    
    # Add the new column to the dataframe
    structures_df['residue_sasa_sum'] = residue_sasa_sum_list
    
    return structures_df



def add_structural_info_to_df(df, structures_data, chain_type='Heavy'):
    """
    Add structural information from structures_results_seq to chain dataframes.
    
    Parameters:
    df: DataFrame with position, amino_acid, region columns
    structures_data: Row from structures_results_seq containing all structural data
    chain_type: 'Heavy' or 'Light'
    """
    chain_letter = 'H' if chain_type == 'Heavy' else 'L'
    
    # 1. Add disulfide bond information
    df['disulfide_bond'] = False
    disulfide_bonds = structures_data['disulfide_bonds']
    if isinstance(disulfide_bonds, list):
        for bond in disulfide_bonds:
            for cys_key in ['cys1', 'cys2']:
                if cys_key in bond:
                    cys_info = bond[cys_key]
                    # Check if it's for this chain
                    if cys_info.endswith(f'_{chain_letter}'):
                        # Extract position (e.g., 'H_21' -> 21)
                        try:
                            pos = int(cys_info.split('_')[1])
                            df.loc[df['position'] == pos, 'disulfide_bond'] = True
                        except (ValueError, IndexError):
                            continue
    
    # 2. Add SAP (Spatial Aggregation Propensity) per position
    df['sap'] = np.nan
    residue_sap = structures_data['residue_sap']
    if isinstance(residue_sap, dict):
        for key, value in residue_sap.items():
            if key.endswith(f'_{chain_letter}'):
                parts = key.split('_')
                if len(parts) >= 3:
                    try:
                        pos = int(parts[1]) + 1  # Convert 0-based to 1-based
                        df.loc[df['position'] == pos, 'sap'] = value
                    except ValueError:
                        continue
    
    # 3. Add high SAP flag
    df['high_sap'] = False
    high_sap_residues = structures_data['high_sap_residues']
    if isinstance(high_sap_residues, list):
        for residue in high_sap_residues:
            if residue.endswith(f'_{chain_letter}'):
                parts = residue.split('_')
                if len(parts) >= 3:
                    try:
                        pos = int(parts[1]) + 1  # Convert 0-based to 1-based
                        df.loc[df['position'] == pos, 'high_sap'] = True
                    except ValueError:
                        continue
    
    # 4. Add sidechain SASA per position
    df['sidechain_sasa'] = np.nan
    sidechain_sasa = structures_data['sidechain_sasa']
    if isinstance(sidechain_sasa, dict):
        for key, value in sidechain_sasa.items():
            if key.endswith(f'_{chain_letter}'):
                parts = key.split('_')
                if len(parts) >= 3:
                    try:
                        pos = int(parts[1]) + 1  # Convert 0-based to 1-based
                        df.loc[df['position'] == pos, 'sidechain_sasa'] = value
                    except ValueError:
                        continue
    
    # 5. Add buried residues flag
    df['buried'] = False
    buried_residues = structures_data['buried_residues']
    if isinstance(buried_residues, list):
        for residue in buried_residues:
            if residue.endswith(f'_{chain_letter}'):
                parts = residue.split('_')
                if len(parts) >= 3:
                    try:
                        pos = int(parts[1]) + 1  # Convert 0-based to 1-based
                        df.loc[df['position'] == pos, 'buried'] = True
                    except ValueError:
                        continue
    
    # 6. Add pKa values
    df['pka'] = None
    residue_pka = structures_data['residue_pka']
    if isinstance(residue_pka, dict):
        for key, value in residue_pka.items():
            if key.endswith(f'_{chain_letter}'):
                parts = key.split('_')
                if len(parts) >= 3:
                    try:
                        pos = int(parts[1]) + 1  # Convert 0-based to 1-based
                        df.loc[df['position'] == pos, 'pka'] = value
                    except ValueError:
                        continue
    
    # 7. Add pKa shifts
    df['pka_shift'] = None
    pka_shifts = structures_data['pka_shifts']
    if isinstance(pka_shifts, dict):
        for key, value in pka_shifts.items():
            if key.endswith(f'_{chain_letter}'):
                parts = key.split('_')
                if len(parts) >= 3:
                    try:
                        pos = int(parts[1]) + 1  # Convert 0-based to 1-based
                        df.loc[df['position'] == pos, 'pka_shift'] = value
                    except ValueError:
                        continue
    
    # 8. Add residue SASA sum per position
    df['residue_sasa_sum'] = np.nan
    residue_sasa_sum = structures_data['residue_sasa_sum']
    if isinstance(residue_sasa_sum, dict):
        # This one is already position-based (1-based indexing)
        for pos, value in residue_sasa_sum.items():
            # Make sure position exists in dataframe
            if (df['position'] == pos).any():
                df.loc[df['position'] == pos, 'residue_sasa_sum'] = value
            else:
                print(f"Warning: Position {pos} in SASA data not found in dataframe")
    
    # Reorder columns for better readability
    base_cols = ['position', 'amino_acid', 'region']
    structural_cols = ['disulfide_bond', 'sap', 'high_sap', 'sidechain_sasa', 
                      'buried', 'pka', 'pka_shift', 'residue_sasa_sum']
    hydro_cols = [col for col in df.columns if col.startswith('hydrophobicity') or col == 'charge_sign']
    
    # Get any remaining columns
    other_cols = [col for col in df.columns if col not in base_cols + structural_cols + hydro_cols]
    
    # New column order
    new_order = base_cols + structural_cols + hydro_cols + other_cols
    available_cols = [col for col in new_order if col in df.columns]
    df = df[available_cols]
    
    return df

# Test antibody sequences (based on therapeutic antibodies)
HEAVY_SEQUENCE = (
    "QVQLVQSGAEVKKPGASVKVSCKASGGTFSSYAISWVRQAPGQGLEWMGGIIPIFGTANYAQKFQGRVTITADESTSTAYMELSSLRSEDTAVYYCARSHYGLDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYASTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK"
)
# Light chain: Includes realistic VL domain + human kappa constant region  
LIGHT_SEQUENCE = (
    "DIQMTQSPSSLSASVGDRVTITCRASHSISSYLAWYQQKPGKAPKLLIYAASSLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQSYSTPLTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC"
)

# Extract the liabilities list from the DataFrame
# Assuming 'liabilities' is a DataFrame with a column called 'liabilities' containing the list
liabilities_list = liabilities['liabilities'].iloc[0]  # Get the list from the first row
structures_results_seq = add_residue_sasa_sum_column(structure_results_seq)
structures_data = structures_results_seq.iloc[0]


# Create comprehensive dataframes for both chains
df_heavy = create_comprehensive_df(annotated_H, heavy_profiles, 'Heavy')
df_light = create_comprehensive_df(annotated_L, light_profiles, 'Light')

# Add liability columns to both dataframes
df_heavy = add_liability_columns(df_heavy, 'Heavy', liabilities_list)
df_light = add_liability_columns(df_light, 'Light', liabilities_list)


# Add structural information to both dataframes
df_heavy = add_structural_info_to_df(df_heavy, structures_data, 'Heavy')
df_light = add_structural_info_to_df(df_light, structures_data, 'Light')


In [ ]:
df_heavy

In [21]:
structures_results_seq.drop("residue_sasa", axis =1 ).style

,disulfide_bonds,residue_sap,high_sap_residues,sidechain_sasa,relative_sasa,buried_residues,residue_pka,pka_shifts,charge_profile,stability_profile,residue_sasa_sum
0,"[{'cys1': 'H_21', 'cys2': 'H_95', 'distance': 2.0656693}]","{'ASP_0_H': 0.029531839957845273, 'VAL_1_H': 0.22409270522155714, 'GLN_2_H': 0.18572525945039642, 'LEU_3_H': 0.0, 'VAL_4_H': 0.4981468204438225, 'GLU_5_H': 0.0, 'SER_6_H': 0.2780227344662963, 'GLY_7_H': 0.0, 'GLY_8_H': 0.0, 'GLY_9_H': 0.0, 'LEU_10_H': 0.6865533901431893, 'VAL_11_H': 0.09822566506655421, 'GLN_12_H': 0.20162946633482023, 'PRO_13_H': 0.2863777431893514, 'GLY_14_H': 0.0, 'GLY_15_H': 0.0, 'SER_16_H': 0.19476138317977099, 'LEU_17_H': 0.2354738067723094, 'ARG_18_H': 0.0, 'LEU_19_H': 0.0, 'SER_20_H': 0.14553212432330892, 'CYS_21_H': 0.0, 'ALA_22_H': 0.4282162702222434, 'ALA_23_H': 0.0, 'SER_24_H': 0.2421952094095717, 'GLY_25_H': 0.0, 'PHE_26_H': 0.06106924408253099, 'SER_27_H': 0.2876869361646816, 'VAL_28_H': 0.0076586118157719015, 'THR_29_H': 0.21001522490413163, 'GLY_30_H': 0.0, 'THR_31_H': 0.06357955871609518, 'TYR_32_H': 0.24389181372243268, 'MET_33_H': 0.0005040298617783074, 'GLY_34_H': 0.0, 'TRP_35_H': 0.001537711650332451, 'PHE_36_H': 0.06081280766240419, 'ARG_37_H': 0.0, 'GLN_38_H': 0.14946509296938285, 'ALA_39_H': 0.1880053826712512, 'PRO_40_H': 0.6340418917602598, 'GLY_41_H': 0.0, 'LYS_42_H': 0.21728542847198618, 'GLU_43_H': 0.0415650842337965, 'ARG_44_H': 0.0, 'GLU_45_H': 0.011945227169948624, 'PHE_46_H': 0.37395178099806214, 'VAL_47_H': 0.0005809268589001226, 'ALA_48_H': 0.0, 'GLY_49_H': 0.0, 'ILE_50_H': 0.027419818660127056, 'GLY_51_H': 0.0, 'GLY_52_H': 0.0, 'SER_53_H': 0.3178782779357969, 'ASP_54_H': 0.019662618722254388, 'HIS_55_H': 0.07641461546086023, 'SER_56_H': 0.14847577740999535, 'THR_57_H': 0.16951896322706725, 'TYR_58_H': 0.655750275403068, 'TYR_59_H': 0.15640147079557623, 'VAL_60_H': 0.25372028498510685, 'ASP_61_H': 0.024816578916671876, 'SER_62_H': 0.1829646406719026, 'LEU_63_H': 0.0011641868374241944, 'LYS_64_H': 0.178119235248267, 'GLY_65_H': 0.0, 'ARG_66_H': 0.0, 'PHE_67_H': 0.0, 'THR_68_H': 0.34619720417253225, 'ILE_69_H': 0.0, 'SER_70_H': 0.19861268567187926, 'ARG_71_H': 0.0, 'ASP_72_H': 0.009402228019357197, 'ASN_73_H': 0.08569937718640665, 'SER_74_H': 0.2790663444211113, 'LYS_75_H': 0.19337026064324583, 'ASN_76_H': 0.055684677331912985, 'THR_77_H': 0.06926024826284065, 'VAL_78_H': 0.0007893581497210229, 'TYR_79_H': 0.1483011500909777, 'LEU_80_H': 0.0, 'GLN_81_H': 0.09880122652151767, 'MET_82_H': 0.0, 'ASN_83_H': 0.11879385386752851, 'SER_84_H': 0.2648207322760928, 'LEU_85_H': 0.0, 'ARG_86_H': 0.0, 'PRO_87_H': 0.4895567783675192, 'GLU_88_H': 0.025122092894593565, 'ASP_89_H': 3.8364517060777035e-05, 'THR_90_H': 0.20117250411780097, 'ALA_91_H': 0.0073012564500125055, 'VAL_92_H': 0.3001034329596522, 'TYR_93_H': 0.002060014850756245, 'TYR_94_H': 0.17120954639395633, 'CYS_95_H': 0.0, 'ALA_96_H': 0.0, 'ARG_97_H': 0.0, 'TRP_98_H': 0.230862485935431, 'ASP_99_H': 0.01643012706060977, 'GLN_100_H': 0.10628960825278448, 'LEU_101_H': 0.8935103553607989, 'GLU_102_H': 0.03920007734384745, 'TYR_103_H': 0.8587283646797502, 'TYR_104_H': 0.9010830749722957, 'TYR_105_H': 0.6090069925713347, 'GLY_106_H': 0.0, 'MET_107_H': 0.20342289175972586, 'ASP_108_H': 0.007216056320452706, 'VAL_109_H': 0.24887618827006663, 'TRP_110_H': 0.2535851457140606, 'GLY_111_H': 0.0, 'GLN_112_H': 0.1386957783907642, 'GLY_113_H': 0.0, 'THR_114_H': 0.04973533810861082, 'THR_115_H': 0.3261708106783313, 'VAL_116_H': 0.0, 'THR_117_H': 0.2417628878717819, 'VAL_118_H': 0.0, 'SER_119_H': 0.21963401063834873, 'SER_120_H': 0.41955348853812774}","['LEU_10_H', 'PRO_40_H', 'TYR_58_H', 'LEU_101_H', 'TYR_103_H', 'TYR_104_H', 'TYR_105_H']","{'ASP_0_H': 111.79910841184281, 'VAL_1_H': 31.78042001323901, 'GLN_2_H': 106.55154327034695, 'LEU_3_H': 0.0, 'VAL_4_H': 70.64627635385119, 'GLU_5_H': 0.0, 'SER_6_H': 56.53387079676777, 'GLY_7_H': 0, 'GLY_8_H': 0, 'GLY_9_H': 0, 'LEU_10_H': 99.74317545028308, 'VAL_11_H': 13.930185227620415

In [22]:
structure_results_seq


,disulfide_bonds,residue_sap,high_sap_residues,residue_sasa,sidechain_sasa,relative_sasa,buried_residues,residue_pka,pka_shifts,charge_profile,stability_profile,residue_sasa_sum
0,"[{'cys1': 'H_21', 'cys2': 'H_95', 'distance': ...","{'ASP_0_H': 0.029531839957845273, 'VAL_1_H': 0...","[LEU_10_H, PRO_40_H, TYR_58_H, LEU_101_H, TYR_...","{'ASP_0_H': {'N': 40.450005754993995, 'CA': 9....","{'ASP_0_H': 111.79910841184281, 'VAL_1_H': 31....",{},"[LEU_3_H, GLU_5_H, LEU_19_H, CYS_21_H, VAL_28_...","0 {'ASP_54_H': 2.55, 'ASP_61_H': 2.76, 'ASP...","0 {'ASP_54_H': -1.25, 'ASP_61_H': -1.04, 'A...","0 {0.0: {'folded': 11.99, 'unfolded': 12.0}...","0 {0.0: 11.82, 1.0: 11.76, 2.0: 11.18, 3.0:...","{1: 111.79910841184281, 2: 31.78042001323901, ..."


In [23]:


# Apply the function to your dataframe
structures_results_seq = add_residue_sasa_sum_column(structure_results_seq)


In [24]:
structures_results_seq

,disulfide_bonds,residue_sap,high_sap_residues,residue_sasa,sidechain_sasa,relative_sasa,buried_residues,residue_pka,pka_shifts,charge_profile,stability_profile,residue_sasa_sum
0,"[{'cys1': 'H_21', 'cys2': 'H_95', 'distance': ...","{'ASP_0_H': 0.029531839957845273, 'VAL_1_H': 0...","[LEU_10_H, PRO_40_H, TYR_58_H, LEU_101_H, TYR_...","{'ASP_0_H': {'N': 40.450005754993995, 'CA': 9....","{'ASP_0_H': 111.79910841184281, 'VAL_1_H': 31....",{},"[LEU_3_H, GLU_5_H, LEU_19_H, CYS_21_H, VAL_28_...","0 {'ASP_54_H': 2.55, 'ASP_61_H': 2.76, 'ASP...","0 {'ASP_54_H': -1.25, 'ASP_61_H': -1.04, 'A...","0 {0.0: {'folded': 11.99, 'unfolded': 12.0}...","0 {0.0: 11.82, 1.0: 11.76, 2.0: 11.18, 3.0:...","{1: 111.79910841184281, 2: 31.78042001323901, ..."


In [25]:
now use the df to add info /cols to the one with out of liability, so precise disulfide bond cysteine, but also the sap per position, the one with high sap with True and others false, the side chain sasa per position, the buried residues as true others as false, the pka per residues , if no pka then None, same for pka shift and the charge, and the residue_sasa_sum per position 

SyntaxError: invalid syntax (3369988904.py, line 1)

In [ ]:
disulfide_bonds	residue_sap	high_sap_residues	residue_sasa	sidechain_sasa	relative_sasa	buried_residues	residue_pka	pka_shifts	charge_profile	stability_profile	residue_sasa_sum
0	[{'cys1': 'H_21', 'cys2': 'H_95', 'distance': ...	{'ASP_0_H': 0.029531839957845273, 'VAL_1_H': 0...	[LEU_10_H, PRO_40_H, TYR_58_H, LEU_101_H, TYR_...	{'ASP_0_H': {'N': 40.450005754993995, 'CA': 9....	{'ASP_0_H': 111.79910841184281, 'VAL_1_H': 31....	{}	[LEU_3_H, GLU_5_H, LEU_19_H, CYS_21_H, VAL_28_...	0 {'ASP_54_H': 2.55, 'ASP_61_H': 2.76, 'ASP...	0 {'ASP_54_H': -1.25, 'ASP_61_H': -1.04, 'A...	0 {0.0: {'folded': 11.99, 'unfolded': 12.0}...	0 {0.0: 11.82, 1.0: 11.76, 2.0: 11.18, 3.0:...	{1: 111.79910841184281, 2: 31.78042001323901, ...


### 2. Chain annotation

In [ ]:
peptide_results
heavy_valid, light_valid
cdrs_H, cdrs_L

### 3. Antibody annotation

In [ ]:
sequence_results
peptide_results
heavy_valid, light_valid
cdrs_H, cdrs_L